Como cada uno de los archivos resultado, de las 12 instancaias, pesaba aproximadamente 10GB, los procesamos en cada instancia, y guardamos una lista, en binario, con los resultados de los campos de la tabla de specs, y subimos esta lista al bucket de Google Cloud Storage.

De esta forma, permitiamos descargar luego estas listas procesadas por cada instancia, que tendrán un tamaño mucho menor.

La función utilizada para obtener la tabla de specs, es muy similar a la vista anteriormente. De todas formas, vamos a ver un ejemplo.

Nos autenticamos con Google Cloud Storage, para obtener el dataset. Lo cargamos en un dataset de pandas, obtenemos el primer permalink y realizamos la request.

In [4]:
project_id = 'cryptic-opus-335323'
bucket_name = 'bdm-unlu'
from google.colab import auth
auth.authenticate_user()
!gsutil cp gs://{bucket_name}/attributes/dataset.csv dataset.csv

import pandas as pd
pd.set_option('display.max_colwidth', 150)
data = pd.read_csv("dataset.csv")

permalink = data.head(1).permalink[0]
permalink

import requests

r = requests.get(permalink)

r

Copying gs://bdm-unlu/attributes/dataset.csv...
- [1 files][ 97.5 MiB/ 97.5 MiB]                                                
Operation completed over 1 objects/97.5 MiB.                                     


<Response [200]>

Definimos la función que va a obtener cada par clave valor, de la tabla de specs.

Busca primero el div con la clase ui-pdp-specs, luego la tabla con la clase ui-pdp-specs__table, y finalmente, dentro de este ultimo tag, las claves que son th con la clase ui-pdp-specs__table__column-title y los valores que son span, con la clase andes-table__column--value

In [7]:
from bs4 import BeautifulSoup
def process_result(result):
  dict_result = {}
  soup = BeautifulSoup(result, 'html.parser')
  div = soup.find_all("div", {"class":"ui-pdp-specs"})[0]
  table = div.find_all("div", {"class":"ui-pdp-specs__table"})[0]
  keys = table.find_all("th", {"class":"ui-pdp-specs__table__column-title"})
  values = table.find_all("span", {"class":"andes-table__column--value"})

  for pair in zip(keys, values):
    dict_result[pair[0].text] = pair[1].text
  return dict_result


Y finalmente vemos el diccionario, con los valores deseados.

In [8]:
process_result(r.text)

{'Ambientes': '3',
 'Antigüedad': '10 años',
 'Baños': '3',
 'Cantidad de pisos': '7',
 'Cocheras': '1',
 'Disposición': 'Frente',
 'Dormitorios': '2',
 'Expensas': '20000 ARS',
 'Superficie cubierta': '85 m²',
 'Superficie total': '85 m²'}

El código fuente ejecutado en las 12 instancias, se encuentra en el repositorio, en el path html_parser/source_code/